In [10]:
%%capture
!pip install pydub pyht
!pip install -U g4f[all]

In [139]:

import random
from pydub import AudioSegment
from io import BytesIO
import requests
from tqdm.notebook import tqdm
import IPython
import json

from pyht import Client
from pyht.client import TTSOptions
import re

import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters


import nest_asyncio
import g4f
nest_asyncio.apply()

import shutil
from tqdm.notebook import tqdm
import pickle

In [105]:
|abbreviations = ["etc", "err", "et", "ex", "e.g", "fin", "i.e", "vs", "N.B", "AD", "BC", "AM", "PM", "P.S", "mph", "kph", "ft", "lb", "oz", "pt", "in", "sec", "g", "cm", "qt"]

playht_url = "https://api.play.ht/api/v2/tts"

# GPT

In [12]:
def find_acitve_gpt():
  active_providers = []
  providers = dir(g4f.Provider)
  for provider in providers:
    provider = getattr(g4f.Provider, provider)
    try:
      response = g4f.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": "Tell me your model name"}],
      provider=provider
      )
      print(response)

      if response != '':
        active_providers.append(provider)
    except:
      continue
  return active_providers

In [13]:
def prompt(text):
  response = g4f.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "I need you to translate from Russian to English and rewrite my text for 'speech text' for narrator and to make it more engaging, but you should sound natural and in casual style, also revise text for proper spelling and grammar. Ignore all tables and tasks."},
              {'role':'assistant','content':'I am a professional rewriter and I am ready to assist you with rewriting your text to ensure better flow and increased engagement. Additionally, I will thoroughly review the text to correct any spelling and grammar errors. Thank you for entrusting me with this task.'},
              {'role':'user','content':text}],
    provider=g4f.Provider.Liaobots
)

  return response

# Voice

In [14]:
def get_payload(text):
  payload = {
      "text": text,
      "voice": "s3://voice-cloning-zero-shot/4bc2279c-ec2e-471f-9369-faf5dac29321/original/manifest.json",
      "output_format": "mp3",
      "voice_engine": "PlayHT2.0",
      "temperature": 0.333,
      "seed": 42}
  return payload

In [15]:
def get_headers(api,user_id):
  headers = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    "AUTHORIZATION": api,
    "X-USER-ID": user_id}
  return headers

In [109]:
def api_checker(text):

  for API in APIs:
    if len(text) + 1 < APIs[API]:
      api = API
      url = "https://api.play.ht/api/v2/tts"

      payload = {
          "text": "h",
          "voice": "s3://voice-cloning-zero-shot/d9ff78ba-d016-47f6-b0ef-dd630f59414e/female-cs/manifest.json",
          "output_format": "mp3",
          "voice_engine": "PlayHT2.0"
      }
      headers = {
          "accept": "text/event-stream",
          "content-type": "application/json",
          "AUTHORIZATION": api,
          "X-USER-ID": user_ids[api]}

      response = requests.post(url, json=payload, headers=headers)

      if response.status_code == 200:
        APIs[api] -= 1
        print(API, APIs[api] - len(text) )
        return api
      else:
        print(f'NOT WORKING {API}, {response.status_code}')
        continue

In [124]:
def TTS(text):
  #название файла
  path = text.split()[0]

  # Создаем токенизатор предложений с нашими параметрами
  punkt_param = PunktParameters()
  punkt_param.abbrev_types = set(abbreviations)
  tokenizer = PunktSentenceTokenizer(punkt_param)
  # Разбиваем текст на предложения
  sentences = tokenizer.tokenize(text)

  api = api_checker(text)


  audio_files = []
  for sentence in tqdm(sentences):


    #создание headers
    user_id = user_ids[api]
    headers = get_headers(api,user_id)

    # создаем настройки для синтеза голоса, параметр только предложение
    payload = get_payload(sentence)
    make_audio = requests.post(playht_url, json=payload, headers=headers)

    audio_url = make_audio.text.split('"url":"')[1].split('","duration"')[0]
    APIs[api] -= len(sentences)

    download_audio = requests.get(audio_url)
    audio_bytes = BytesIO(download_audio.content)
    audio_file = AudioSegment.from_file(audio_bytes)
    audio_files.append(audio_file)

  return merge_audio(audio_files,path)

In [125]:
def merge_audio(audio_files,path):

  # Создание сегмента тишины продолжительностью 1 секунда
  silence = AudioSegment.silent(duration=1)

  # Создание списка сегментов аудиофайлов
  audio_segments = [AudioSegment.from_mono_audiosegments(file) for file in audio_files]

  # Создание списка для склеенных сегментов
  merged_segments = []

  # Добавление первого сегмента
  merged_segments.append(audio_segments[0])

  # Добавление паузы и остальных аудиофайлов
  for segment in tqdm(audio_segments[1:]):
    merged_segments.extend([silence, segment])

  # Склеивание аудиофайлов
  merged_audio = sum(merged_segments)

  path = f"{path}.wav"
  # Сохранение склеенного аудиофайла
  merged_audio.export(path, format="wav")
  return path

In [19]:
abbreviations = ["etc", "err", "et", "ex", "e.g", "fin", "i.e", "vs", "N.B", "AD", "BC", "AM", "PM", "P.S", "mph", "kph", "ft", "lb", "oz", "pt", "in", "sec", "g", "cm", "qt"]

In [95]:
keys = '''fahoweg765@mnsaf.com;Vn8hNRq*;0FzMG5HDWlfA7y8cuzrUr4Kjjlz2;1fb68c8352e7401ab9d23736be63e7fb
lafiw72077@mnsaf.com;u3Q=9z!%;cohjCRHXy7PFfPL9TBy0HlslDcf2;ff5b56b781154f4b85aaa347bc2ddbfd
cofopey509@nimadir.com;.5]^WUPU;gBFVYS1MuHVRz850luXkzph4l1D2;3dc9df0b56ff4380af875a5adc7c2cfc
wogaya1695@mnsaf.com;99~YmHNm;EOupbZitomf2FZezAi4rq9YBbv62;18c08454727f4733b4986c129c0ce416
kitici5534@mnsaf.com;~.wmyD0U;r6WO17KzCkhKHewOL7Hs5PSBKA52;979a04ffed504b96bd08e34145d8c383'''.split('\n')


In [134]:
!git clone https://github.com/PaleBill/stepik_abuse
!cd stepik_abuse

Cloning into 'stepik_abuse'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (26/26), 50.36 KiB | 6.29 MiB/s, done.
Resolving deltas: 100% (5/5), done.


# Playground


In [96]:
keys

['fahoweg765@mnsaf.com;Vn8hNRq*;0FzMG5HDWlfA7y8cuzrUr4Kjjlz2;1fb68c8352e7401ab9d23736be63e7fb',
 'lafiw72077@mnsaf.com;u3Q=9z!%;cohjCRHXy7PFfPL9TBy0HlslDcf2;ff5b56b781154f4b85aaa347bc2ddbfd',
 'cofopey509@nimadir.com;.5]^WUPU;gBFVYS1MuHVRz850luXkzph4l1D2;3dc9df0b56ff4380af875a5adc7c2cfc',
 'wogaya1695@mnsaf.com;99~YmHNm;EOupbZitomf2FZezAi4rq9YBbv62;18c08454727f4733b4986c129c0ce416',
 'kitici5534@mnsaf.com;~.wmyD0U;r6WO17KzCkhKHewOL7Hs5PSBKA52;979a04ffed504b96bd08e34145d8c383']

In [97]:
APIs = {}
user_ids = {}

In [98]:
keys = [key.split(';')[2:] for key in keys]

In [99]:
for key in keys:
  APIs[key[1]] = 12500
  user_ids[key[1]] = key[0]

In [100]:
APIs['1fb68c8352e7401ab9d23736be63e7fb'] = 10000

In [101]:
APIs

{'1fb68c8352e7401ab9d23736be63e7fb': 10000,
 'ff5b56b781154f4b85aaa347bc2ddbfd': 12500,
 '3dc9df0b56ff4380af875a5adc7c2cfc': 12500,
 '18c08454727f4733b4986c129c0ce416': 12500,
 '979a04ffed504b96bd08e34145d8c383': 12500}

In [102]:
user_ids

{'1fb68c8352e7401ab9d23736be63e7fb': '0FzMG5HDWlfA7y8cuzrUr4Kjjlz2',
 'ff5b56b781154f4b85aaa347bc2ddbfd': 'cohjCRHXy7PFfPL9TBy0HlslDcf2',
 '3dc9df0b56ff4380af875a5adc7c2cfc': 'gBFVYS1MuHVRz850luXkzph4l1D2',
 '18c08454727f4733b4986c129c0ce416': 'EOupbZitomf2FZezAi4rq9YBbv62',
 '979a04ffed504b96bd08e34145d8c383': 'r6WO17KzCkhKHewOL7Hs5PSBKA52'}

In [47]:
with open('/content/291011.json', 'r', encoding='UTF-8') as f:
    data = json.load(f)
data[0];

In [51]:
for lesson in data[0:1]:
  for step in lesson['steps'][1:2]:
    text = step['text']['text']
    print(f'text)

Связь «один ко многим»
Рассмотрим таблицу book(в ней столбец author переименован в name_author):

[TABLE]

В этой таблице фамилии авторов повторяются для нескольких книг. А что, если придется вместо инициалов для каждого автора хранить его полное имя и отчество? Тогда, если в таблице содержится информация о 50 книгах Достоевского, придется 50 раз исправлять «Ф.М.» на «Федор Михайлович». При этом, если в некоторых записях использовать «Фёдор Михайлович» (c буквой ё), то мы вообще получим двух разных авторов... 
Чтобы устранить эту проблему в реляционных базах данных создается новая таблица author,  в которой перечисляются все различные авторы, а затем эта таблица связывается с таблицей book. При этом такая связь называется «один ко многим», таблица author называется главной, таблица book – связанной или подчиненной. 
Связь «один ко многим» имеет место, когда одной записи главной таблицы соответствует несколько записей связанной таблицы, а каждой записи связанной таблицы соответствует то

In [128]:
text


'"Many-to-One" Relationship\nLet\'s take a look at the table \'book\' (where the column \'author\' has been renamed to \'name_author\'):\n\n[REMOVED TABLE]\n\nIn this table, authors\' last names are repeated for several books. But what if we need to store the full name and middle name instead of initials for each author?'

In [53]:
response = prompt(text)

In [115]:
print(response[:308])

"Many-to-One" Relationship
Let's take a look at the table 'book' (where the column 'author' has been renamed to 'name_author'):

[REMOVED TABLE]

In this table, authors' last names are repeated for several books. But what if we need to store the full name and middle name instead of initials for each author?


In [103]:
api = api_checker(text)

1fb68c8352e7401ab9d23736be63e7fb


In [133]:
APIs

{'1fb68c8352e7401ab9d23736be63e7fb': 5086,
 'ff5b56b781154f4b85aaa347bc2ddbfd': 12500,
 '3dc9df0b56ff4380af875a5adc7c2cfc': 12500,
 '18c08454727f4733b4986c129c0ce416': 12500,
 '979a04ffed504b96bd08e34145d8c383': 12500}

In [131]:
len(response)

2182

In [132]:
5107 - 2182

2925

In [130]:
audio = TTS(response)

1fb68c8352e7401ab9d23736be63e7fb 3228


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [127]:
IPython.display.Audio(audio)

In [138]:
def save():

  with open('apis.pkl', "wb") as file:
    # Use pickle to dump the list to the file
     pickle.dump(APIs, file)
  with open('user_ids.pkl', "wb") as file:
    # Use pickle to dump the list to the file
      pickle.dump(user_ids, file)

In [136]:
APIs['1fb68c8352e7401ab9d23736be63e7fb'] = 2694

In [137]:
APIs

{'1fb68c8352e7401ab9d23736be63e7fb': 2694,
 'ff5b56b781154f4b85aaa347bc2ddbfd': 12500,
 '3dc9df0b56ff4380af875a5adc7c2cfc': 12500,
 '18c08454727f4733b4986c129c0ce416': 12500,
 '979a04ffed504b96bd08e34145d8c383': 12500}

error: remote origin already exists.
